<a href="https://colab.research.google.com/github/HyoJuJang/traffic/blob/master/make_train_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BASELINE

### Library

In [ ]:
# 데이터 보기
import pandas as pd
import numpy as np
#from glob import glob

# 파일경로 설정
import os
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = '/content/drive/MyDrive/2021_2/dacon/go_traffic_file'


train_path = data_path + '/train'
#test_path = data_path + '/test'

new_image_directory = data_path + '/new_images'
new_train_image_directory = new_image_directory + '/train'
#new_test_image_directory = new_image_directory + '/test'

action_information = pd.read_csv(data_path + '/action_information.csv')
#sample_submission = pd.read_csv(data_path + '/sample_submission.csv')

classes = pd.get_dummies(action_information[['Label']], columns = ['Label']).to_numpy()

# Make train data as arrays

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image

## convert & save

In [ ]:
new_train_image_directories = sorted(glob(new_train_image_directory + '/*'), key = lambda x : int(x.split('/')[-1].split('_')[-1]))

In [ ]:
train_answer = []
train_image_directories = sorted(glob(train_path + '/*'), key = lambda x : int(x.split('_')[-1]))
for train_image_directory in train_image_directories:
  json_path = glob(train_image_directory + '/*.json')[0]
  js = json.load(open(json_path))
  action = js.get('action')
  train_answer.append(action)

In [ ]:
len(new_train_image_directories), len(train_answer)

(142, 142)

In [ ]:
# 33번 file이 잘못 annotation 되었다고 함
train_answer[33] = 5
train_answer[30:35]

[1, 5, 5, 5, 5]

In [ ]:
new_train_image_directories = new_train_image_directories[:41]
train_answer = train_answer[:41]
new_train_image_directories

In [ ]:
tf.config.set_soft_device_placement(True)
images = []
targets = []
stratify = []
for num, (new_train_image_directory, action) in tqdm(enumerate(zip(new_train_image_directories, train_answer)), total = len(new_train_image_directories)) : 
    image_paths = sorted(glob(new_train_image_directory + '/*'), key = lambda x : int(x.split('/')[-1].replace('.jpg','')))
    image_len = len(image_paths)
    for image_path in image_paths : 
        img = image.load_img(image_path, target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        images.append(img)
        targets.append(classes[action])
        stratify.append(action)

images = np.array(images)
targets = np.array(targets)

In [ ]:
import pickle

In [ ]:
# np, pickle 쓰기 ###### 주의 : 덮어쓰여짐 ######
# 용량이 너무 커서 파일을 총 4개로 나눠서 저장함(0:41, 41:91, 91:121, 121:142) -> images는 2개로 합침(img_n1 0:91, img_n2 91:142)
# images는 np로 target(첫번째파일은 pickle), stratify는 pickle로

#np.save('/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_img_4', images)

#with open ("/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_tar_2", "wb") as file:
  #pickle.dump(targets, file)

#with open ("/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_stratify_2", "wb") as file:
  #pickle.dump(stratify, file)

## load & reconvert : 최종 train data 만들기

###images

In [ ]:
# 읽기
img_n1 = np.load('/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_array/train_img_n1.npy')
img_n2 = np.load('/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_array/train_img_n2.npy')

In [ ]:
img_n1 = img_n1.astype('float16')

In [ ]:
# 합치기
img = np.concatenate([img_n1, img_n2], axis=0)

In [ ]:
# 저장
np.save('/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_img', img)

### target, stratify

In [ ]:
import pickle

In [ ]:
with open("/content/drive/MyDrive/2021_2/dacon/go_traffic_file/array_tar_41", "rb" ) as file:
    targets1 = pickle.load(file)
with open("/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_tar_2", "rb" ) as file:
    targets2 = pickle.load(file)
with open("/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_tar_3", "rb" ) as file:
    target3 = pickle.load(file)
with open("/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_tar_4", "rb" ) as file:
    targets4 = pickle.load(file)

In [ ]:
tar = np.concatenate([targets1, targets2, target3, targets4], axis=0)

In [ ]:
tar.shape

(22218, 6)

In [ ]:
with open('/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_array/tar', "wb") as file:
  pickle.dump(tar, file)

In [ ]:
with open ("/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_stratify_1", "rb") as file:
  stratify1 = pickle.load(file)
with open ("/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_stratify_2", "rb") as file:
  stratify2 = pickle.load(file)
with open ("/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_stratify_3", "rb") as file:
  stratify3 = pickle.load(file)
with open ("/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_stratify_4", "rb") as file:
  stratify4 = pickle.load(file)

In [ ]:
stra = np.concatenate([stratify1, stratify2, stratify3, stratify4], axis=0)

In [ ]:
stra = list(stra)

In [ ]:
with open ('/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_array/stra', "wb") as file:
  pickle.dump(stra, file)

# Save files

In [ ]:
import numpy as np
import pandas as pd
import pickle

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
images = np.load('/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_array/img_float16.npy')
with open('/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_array/tar', 'rb') as file:
  targets = pickle.load(file)
with open('/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_array/stra', 'rb') as file:
  stratify = pickle.load(file)

In [ ]:
train_idx, valid_idx = train_test_split(range(images.shape[0]), test_size = 0.2, stratify = stratify)

X_train = images[train_idx]
y_train = targets[train_idx]

X_valid = images[valid_idx]
y_valid = targets[valid_idx]

print('X_train shape : ', X_train.shape)
print('X_valid shape : ', X_valid.shape)
print('y_train shape : ', y_train.shape)
print('y_valid shape : ', y_valid.shape)

X_train shape :  (17774, 224, 224, 3)
X_valid shape :  (4444, 224, 224, 3)
y_train shape :  (17774, 6)
y_valid shape :  (4444, 6)


In [ ]:
np.save('/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_array/train_idx',train_idx)

In [ ]:
np.save('/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_array/valid_idx',valid_idx)

In [ ]:
np.save('/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_array/train_X', X_train)

In [ ]:
np.save('/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_array/train_y', y_train)

In [ ]:
np.save('/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_array/valid_X', X_valid)
np.save('/content/drive/MyDrive/2021_2/dacon/go_traffic_file/train_array/valid_y', y_valid)